# Starting Kit - Relevance 

TODOs: 
- Add detailed description of the challenge
- describe your data
- describe how you will evaluate
- provide instructions to the participants about what they should do

***
# Setup
***
`COLAB` determines whether this notebook is running on Google Colab.

In [1]:
COLAB='google.colab' in str(get_ipython())

In [2]:
if COLAB:
    # clone github repo
    !git clone https://github.com/ihsaan-ullah/M1-Challenge-Class-2024.git

    # move to the HEP starting kit folder
    %cd M1-Challenge-Class-2024/Relevance/Starting_Kit/

    !pip install -q --upgrade sentence-transformers transformers

***
# Imports
***

In [3]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from tqdm.notebook import tqdm

from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from scipy.stats import kendalltau

import torch
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")
tqdm.pandas()

***
# Directories
***

In [4]:
root_dir = "./"
# Input data directory to read training data from
input_dir = root_dir + "sample_data/"
# Reference data directory to read test labels from
reference_dir = root_dir + "sample_data/"
# Output data directory to write predictions to
output_dir = root_dir + "sample_result_submission"
# Program directory
program_dir = root_dir + "ingestion_program"
# Score directory
score_dir = root_dir + "scoring_program"
# Directory to read submitted submissions from
submission_dir = root_dir + "sample_code_submission"

***
# Add directories to path
***

In [5]:
sys.path.append(input_dir)
sys.path.append(reference_dir)
sys.path.append(output_dir)
sys.path.append(program_dir)
sys.path.append(submission_dir)

***
# Data
***
1. Load Data
2. Preprocess data


TODOS:
- show data statistics

### ⚠️ Note:
The data used here is sample data is for demonstration only to get a view of what the data looks like.

In [6]:
class Data():

  def __init__(self):

      self.df = None

      print("==========================================")
      print("Data")
      print("==========================================")

  def load_data(self):
    """
      Loads data from csv file
    """
    print("[*] Loading Data")

    # data file path
    data_file = os.path.join(input_dir, 'relevance_sample_data.csv')
    
    # read data
    self.df = pd.read_csv(data_file)


  def _text_to_dict(self, text):
    """
    Converts a text string into a dictionary.

    :param text: A string representation of a dictionary.
    :return: A dictionary object if conversion is successful, otherwise {}.
    """
    try:
      return ast.literal_eval(text)
    except:
      return {}  # Return an empty dictionary in case of an error
    
  def _dict_to_paragraphs(self, dictionary):
    """
    Converts a dictionary into a string of paragraphs.

    :param dictionary: A dictionary.
    :return: A string composed of paragraphs based on the dictionary's key-value pairs.
    """
    text = ''
    for i, (k, v) in enumerate(dictionary.items()):
        text += k.capitalize() + '\n' + v + '\n'
    return text
  
  def transfrom_data(self):

    print("[*] Transforming Data")
    
    # Convert to dictionary
    self.df['most_relevant_dict'] = self.df['most_relevant'].apply(self._text_to_dict)
    self.df['second_most_relevant_dict'] = self.df['second_most_relevant'].apply(self._text_to_dict)
    self.df['second_least_relevant_dict'] = self.df['second_least_relevant'].apply(self._text_to_dict)
    self.df['least_relevant_dict'] = self.df['least_relevant'].apply(self._text_to_dict)


    # Convert from dictionary to text
    self.df['most_relevant_text'] = self.df['most_relevant_dict'].apply(self._dict_to_paragraphs)
    self.df['second_most_relevant_text'] = self.df['second_most_relevant_dict'].apply(self._dict_to_paragraphs)
    self.df['second_least_relevant_text'] = self.df['second_least_relevant_dict'].apply(self._dict_to_paragraphs)
    self.df['least_relevant_text'] = self.df['least_relevant_dict'].apply(self._dict_to_paragraphs)


  def _get_embeddings(self, text1, text2):
    """
    Generates embeddings for two texts.

    :param text1: First text string.
    :param text2: Second text string.
    :return: Tuple of embeddings for text1 and text2.
    """
    embedding1 = self.embeddings_model.encode(text1, convert_to_tensor=True)
    embedding2 = self.embeddings_model.encode(text2, convert_to_tensor=True)
    return embedding1.cpu(), embedding2.cpu()
  
  def prepare_data(self):

    print("[*] Prepare Data for Training")
    
    model_name = 'paraphrase-MiniLM-L6-v2'
    device = "cuda" if torch.cuda.is_available() else "cpu"
    self.embeddings_model = SentenceTransformer(model_name, device=device)

    # Create embeddings for each pair
    self.df['most_relevant_embeddings'] = self.df.progress_apply(lambda row: self._get_embeddings(row['prompt'], row['most_relevant_text']), axis=1)
    self.df['second_most_relevant_embeddings'] = self.df.progress_apply(lambda row: self._get_embeddings(row['prompt'], row['second_most_relevant_text']), axis=1)
    self.df['second_least_relevant_embeddings'] = self.df.progress_apply(lambda row: self._get_embeddings(row['prompt'], row['second_least_relevant_text']), axis=1)
    self.df['least_relevant_embeddings'] = self.df.progress_apply(lambda row: self._get_embeddings(row['prompt'], row['least_relevant_text']), axis=1)

    # Label the Data
    self.df['most_relevant_label'] = 3
    self.df['second_most_relevant_label'] = 2
    self.df['second_least_relevant_label'] = 1
    self.df['least_relevant_label'] = 0

    X = self.df['most_relevant_embeddings'].tolist() + self.df['second_most_relevant_embeddings'].tolist() + self.df['second_least_relevant_embeddings'].tolist() + self.df['least_relevant_embeddings'].tolist()
    y = self.df['most_relevant_label'].tolist() + self.df['second_most_relevant_label'].tolist() + self.df['second_least_relevant_label'].tolist() + self.df['least_relevant_label'].tolist()

    # Convert embeddings from tuples to concatenated arrays
    X = [torch.abs(embeddings[0] - embeddings[1]).numpy() for embeddings in X]

    # Shuffle X and y
    X, y = shuffle(X, y, random_state=42)

    # train test split
    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)


  def get_train_data(self):
    return self.X_train, self.y_train
  
  def get_test_data(self):
    return self.X_test, self.y_test
  
  def show_random_sample(self):
    random_sample_index = np.random.randint(0, len(self.df))

    print("Prompt:\n", self.df.iloc[random_sample_index]['prompt'], "...\n")
    print("Most Relevant Text:\n", self.df.iloc[random_sample_index]['most_relevant_text'][:300], "...\n")
    print("Second Most Relevant Text:\n", self.df.iloc[random_sample_index]['second_most_relevant_text'][:300], "...\n")
    print("Second Least Relevant Text:\n", self.df.iloc[random_sample_index]['second_least_relevant_text'][:300], "...\n")
    print("Least Relevant Text:\n", self.df.iloc[random_sample_index]['least_relevant_text'][:300], "...\n")

In [7]:
# Initilaize data
data = Data()

Data


In [8]:
# load data
data.load_data()

[*] Loading Data


In [9]:
# transform data
data.transfrom_data()

[*] Transforming Data


In [10]:
# prepare data
data.prepare_data()

[*] Prepare Data for Training


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [11]:
# show random sample from data
data.show_random_sample()

Prompt:
 Write a systematic survey or overview about the impact of transcendental empiricism on the study and interpretation of art history, including the critique of traditional linear and teleological conceptions of history and the implications for art historical knowledge, methodology, and politics. ...

Most Relevant Text:
 Title
Istorija (umetnosti) u uslovima transcendentalnog empirizma / (Art) History under the Conditions of Transcendental Empiricism
Abstract
This text deals with a critique of molar conception of history and, particularly, art history. Instead of conceptualizing history as linear and teleological, t ...

Second Most Relevant Text:
 Title
The Work of Art that Stands Alone
Abstract
In ‘From Work to Text’ – Roland Barthes’ classic manifesto that spelled the end of any closed structuralism and opened the text to as many forces, connections and actualisations as possible – Barthes suggests that the closed work is the very death of  ...

Second Least Relevant Text:
 T

In [12]:
# get train test data
# X_train, y_train, X_test, Y_test = data.get_train_test_data()

***
# Visualize
***
TODOs:
- visualize your data

In [13]:
class Visualize():

    def __init__(self, data):

        print("==========================================")
        print("Visualize")
        print("==========================================")
        self.data = data



    def visualize_data(self):
        print("Implement this function")
        pass


In [14]:
# Initilaize Visualize
visualize = Visualize(data=data)
visualize.visualize_data()

Visualize
Implement this function


***
# Import Submission Model
***
We import a class named `Model` from the submission file (`model.py`). This `Model` class has the following methods:
- `init`: initializes classifier
- `fit`: gets train data and labels as input to train the classifier
- `predict`: gets test data and outputs predictions made by the trained classifier


In this example code, the `Model` class implements a Gradient Boosting Classifier model. You can find the code in `M1-Challenge-Class-2024/Relevance/Starting_Kit/sample_code_submission/model.py`. You can modify it the way you want, keeping the required class structure and functions there. More instructions are given inside the `model.py` file. If running in Collab, click the folder icon in the left sidebar to open the file browser.

In [15]:
from model import Model

***
# Program
***
**`Ingestion program`** is responsible to run the submission of a participant on Codabench platform. **`Program`** is a simplified version of the **Ingestion Program** to show to participants how it runs a submission.
1. Train a model on train data
2. Predict using Test data

In [16]:
class Program():

    def __init__(self, data):

        # used to keep object of Model class to run the submission
        self.model = None
        # object of Data class used here to get the train and test sets
        self.data = data

        # results
        self.results = []

        print("==========================================")
        print("Program")
        print("==========================================")
    
    def initialize_submission(self):
        print("[*] Initializing Submmited Model")
        self.model = Model()

    def fit_submission(self):
        print("[*] Calling fit method of submitted model")
        X_train, y_train  = self.data.get_train_data()
        self.model.fit(X_train, y_train)


    def predict_submission(self):
        print("[*] Calling predict method of submitted model")
      
        X_test, _ = self.data.get_test_data()
        self.y_test_hat = self.model.predict(X_test)

       

In [17]:
# Intiialize Program
program = Program(data=data)

Program


In [18]:
# Initialize submitted model
program.initialize_submission()

[*] Initializing Submmited Model
[*] - Initializing Classifier


In [19]:
# Call fit method of submitted model
program.fit_submission()

[*] Calling fit method of submitted model
[*] - Training Classifier on the train set


In [20]:
# Call predict method of submitted model
program.predict_submission()

[*] Calling predict method of submitted model
[*] - Predicting test set using trained Classifier


***
# Score
***

TODOs:
- Explain the evaluation metric


In [21]:
class Score():

    def __init__(self, data, program):

        self.data = data
        self.program = program

        print("==========================================")
        print("Score")
        print("==========================================")

    def compute_scores(self):
        print("[*] Computing scores")

        _, y_test = self.data.get_test_data()
        y_test_hat = self.program.y_test_hat

        # Classification report
        print(classification_report(y_test, y_test_hat))

        k_tau, _ = kendalltau(y_test, y_test_hat)
        print(f"Kendall's Tau: {k_tau}")

In [22]:
# Initialize Score
score = Score(data=data, program=program)

Score


In [23]:
# Compute Score
score.compute_scores()

[*] Computing scores
              precision    recall  f1-score   support

           0       0.51      0.54      0.53        96
           1       0.55      0.51      0.53       107
           2       0.58      0.62      0.60        91
           3       0.77      0.75      0.76       106

    accuracy                           0.60       400
   macro avg       0.60      0.60      0.60       400
weighted avg       0.61      0.60      0.61       400

Kendall's Tau: 0.705678403820533


***
# Submissions
***

### **Unit Testing**

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. All you have to do to make a submission is modify the file <code>model.py</code> in the <code>sample_code_submission/</code> directory, then run this test to make sure everything works fine. This is the actual program that will be run on the server to test your submission.
<br>
Keep the sample code simple.<br>

<code>python3</code> is required for this step

### **Test Ingestion Program**

In [26]:
!python3 $program_dir/ingestion.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


############################################
### Ingestion Program
############################################

[*] Loading Data
[*] Transforming Data
[*] Prepare Data for Training
  0%|          | 0/500 [00:00<?, ?it/s]
[*] Initializing Submmited Model
[*] - Initializing Classifier
[*] Calling fit method of submitted model
Traceback (most recent call last):
  File "/Users/ihsanullah/Desktop/ParisSaclay/Challenge Class/2024/ClassRepo/Relevance/Starting_Kit/./ingestion_program/ingestion.py", line 230, in <module>
    ingestion.fit_submission()
  File "/Users/ihsanullah/Desktop/ParisSaclay/Challenge Class/2024/ClassRepo/Relevance/Starting_Kit/./ingestion_program/ingestion.py", line 185, in fit_submission
    X_train, y_train  = self.data.get_train_data()
AttributeError: 'Ingestion' object has no attribute 'data'


### **Test Scoring Program**

In [ ]:
!python3 $score_dir/score.py

### **Prepare the submission**

In [ ]:
import datetime
from data_io import zipdir
the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")
code_submission = 'Relevance-code_submission_' + the_date + '.zip'
zipdir(code_submission, submission_dir)
print("Submit : " + code_submission + " to the competition")
print("You can find the zip file in `M1-Challenge-Class-2024/Relevance/Starting_Kit/")